# BPR

In [1]:
%reload_ext blackcellmagic
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'pdf'
%config InlineBackend.figure_format = 'retina'
%load_ext line_profiler
%load_ext Cython

In [2]:
import logging

logging.basicConfig(
    format="[%(asctime)s]: %(message)s (%(filename)s:%(lineno)d)",
    level=logging.INFO,
    datefmt="%H.%M.%S",
)

In [3]:
# from bpr import cli
# from bpr.cli import train, Config, train_concurrently
from dataclasses import dataclass
from enum import Enum
from functools import partial
from pprint import pprint
from typing import List, Union, TypeVar, Any, Optional, List
import bpr
import itertools
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import yaml
from multiprocessing import cpu_count
from bpr import plotting
from matplotlib import cm
import glob
import pathlib
from typing import Dict
from pprint import pprint

In [8]:
print("Hello")

Hello


In [9]:


train_df = bpr.ml100k.load(subset="train")
tst_matrix = bpr.ml100k.load_matrix(subset="test")
trn_matrix = bpr.ml100k.load_matrix(subset="train")

BATCH_SIZE = 32
EPOCHS = 200
LR_ITEM = 0.05
LR_USER = 0.05
N_FACTORS = 10
REG = 0.01
SEED = 42
TOP_N = 10
WARM_UP = math.ceil(0.3 * EPOCHS)
WINDOW_SIZE = math.ceil(0.1 * WARM_UP)
WARM_UP = 20
WINDOW_SIZE = 5
q = WINDOW_SIZE

In [10]:
summaries_dict = {}
ptrain = partial(
    bpr.training.train,
    train_df=train_df,
    trn_matrix=trn_matrix,
    tst_matrix=tst_matrix,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    lr_user=LR_USER,
    lr_item=LR_ITEM,
    n_factors=N_FACTORS,
    reg=REG,
    warm_up=WARM_UP,
    window_size=WINDOW_SIZE
)

In [ ]:
summaries_dict["RandomSampler"] = ptrain(
    sampler="RandomSampler",
)

[12.36.29]: sampler=RandomSampler, batch_size=32, epochs=200, lr_item=0.05, lr_user=0.05, n_factors=10, reg=0.01, method=std, rand_option=cached, reverse=True, se_0=100, se_end=10, seed=42, top_n=10, warm_up=20, window_size=5 (training.py:166)
[12.36.29]: seed=42 (_base.py:29)
[12.36.29]: sampler: RandomSampler (training.py:67)
[12.36.31]: Using RandomSampler (training.py:87)
[12.36.31]: Epoch   1 (utils.py:112)
[12.37.00]: Epoch  20 (utils.py:112)
[12.37.31]: Epoch  40 (utils.py:112)
[12.38.01]: Epoch  60 (utils.py:112)
[12.38.32]: Epoch  80 (utils.py:112)
[12.39.03]: Epoch 100 (utils.py:112)


In [7]:
%time
se_params = [
    (1, 10),
    (1, 1e2),
    (1e2, 1),
    (10, 1),
]
method = bpr._types.Methods.GRAD.value
desc = method
for se_0, se_end in se_params:
    summaries_dict["RB_{}_{}_{}".format(int(se_0), int(se_end), desc)] = ptrain(
        sampler="RecencyBias", se_0=se_0, se_end=se_end, method=method
    )
    summaries_dict["OS_{}_{}_{}".format(int(se_0), int(se_end), desc)] = ptrain(
        sampler="OnlineSampler", se_0=se_0, se_end=se_end, method=method
    )

[10.56.35]: sampler=RecencyBias, batch_size=32, epochs=200, lr_item=0.05, lr_user=0.05, n_factors=10, reg=0.01, method=grad, rand_option=cached, reverse=True, se_0=1, se_end=10, seed=42, top_n=10, warm_up=20, window_size=5 (training.py:166)
[10.56.35]: seed=42 (_base.py:29)
[10.56.35]: reverse=True, rand_option=cached (online.py:57)
[10.56.35]: [RecencyBias] warm_up=20, q=5, se_0=1, se_end=10, discount_warm_up=True, reverse=True, rand_option=cached, method=grad (recency.py:81)
[10.56.35]: sampler: RecencyBias (training.py:67)


CPU times: user 338 µs, sys: 22 µs, total: 360 µs
Wall time: 24.1 µs


[10.56.38]: Using RecencyBias (training.py:87)
[10.56.38]: Epoch   1 (utils.py:112)
[10.57.10]: Epoch  20 (utils.py:112)
[10.59.00]: Epoch  40 (utils.py:112)
[11.00.59]: Epoch  60 (utils.py:112)
[11.02.59]: Epoch  80 (utils.py:112)
[11.04.58]: Epoch 100 (utils.py:112)
[11.06.58]: Epoch 120 (utils.py:112)
[11.08.57]: Epoch 140 (utils.py:112)
[11.10.57]: Epoch 160 (utils.py:112)
[11.12.55]: Epoch 180 (utils.py:112)
[11.14.55]: Epoch 200 (utils.py:112)
[11.15.01]: Finished in 1105 seconds (training.py:135)
[11.15.01]: sampler=OnlineSampler, batch_size=32, epochs=200, lr_item=0.05, lr_user=0.05, n_factors=10, reg=0.01, method=grad, rand_option=cached, reverse=True, se_0=1, se_end=10, seed=42, top_n=10, warm_up=20, window_size=5 (training.py:166)
[11.15.01]: seed=42 (_base.py:29)
[11.15.01]: reverse=True, rand_option=cached (online.py:57)
[11.15.01]: sampler: OnlineSampler (training.py:67)
[11.15.05]: Using OnlineSampler (training.py:87)
[11.15.05]: Epoch   1 (utils.py:112)
[11.15.38]: Epoc

KeyboardInterrupt: 

In [ ]:
plotting.plot_gains(summaries_dict)

In [ ]:
summaries_dict['RandomSampler'][1]

In [ ]:
print(sorted(summaries_dict.keys()))
exclude = None
bpr.plotting.plot_convergence_quadrant(
    summaries_dict, exclude=exclude, fontsize="medium", warm_up=WARM_UP, figsize=(15, 5.5)
)

## Train using CLI tool

In [ ]:
bpr.cli

In [ ]:
workers = math.ceil(cpu_count() * 0.5)
configurations = bpr.cli.Config.parse("../config.yaml")
config_filenames = sorted(set([c.filename() for c in configurations]))
pprint(config_filenames[:10])
# train_concurrently(data="../data/", workers=workers, configurations=configurations)

In [ ]:
pprint(json.load(open("processed.json")))

In [ ]:
!ls 

In [ ]:
!ls ../data/outputs

In [ ]:
output_dir = pathlib.Path("../data/outputs")

In [ ]:
def parse(fname: pathlib.Path):
    try:
        dataset, sampler, method, rand_option, others = fname.name.split("-")
    except ValueError:
        print(fname.name)
        dataset, sampler, others = fname.name.split("-")
        method = 'default'
        rand_option = 'default'

    def fname_to_dict() -> Dict:
        parts = dict()
        prefixes = {
            "bs": (int, "batch_size"),
            "ep": (int, "epochs"),
            "li": (float, "lr_item"),
            "lu": (float, "lr_user"),
            "nf": (int, "n_factors"),
            "rg": (float, "reg"),
            "rv": (bool, "reverse"),
            "wu": (int, "warm_up"),
            "se0": (int, "se_0"),
            "send": (int, "se_end"),
            "ws": (int, "window_size"),
        }
        for prefix, class_and_alias in prefixes.items():
            if prefix not in others:
                continue
            cls, alias = class_and_alias

            pattern = r"{}(\d+).*"
            if prefix == "rv":
                pattern = r"{}(\w).*"

            parts[alias] = re.findall(pattern.format(prefix), others)[0]

            if cls is float and parts[alias].startswith("0"):
                z = list(parts[alias])
                z.insert(1, ".")
                parts[alias] = "".join(z)
            parts[alias] = cls(parts[alias])

        return parts
    sampler_lookups = {'rec': 'RecencyBias', 'ran': 'RandomSampler', 'onl': 'OnlineSampler'}
    parsed = fname_to_dict()
    parsed['dataset'] = dataset.replace("mk", "ml-100k")
    parsed['method'] = method.replace("mn", 'mean')
    parsed['rand_option'] = rand_option.replace("ca", "cached")
    parsed['sampler'] = sampler_lookups.get(sampler, sampler)
    df = pd.read_feather(fname.as_posix())
    df['experiment'] = fname.name.split(".")[0]
    return df, parsed


def _fn():
    for idx, fname in enumerate(output_dir.resolve().glob("*.feather")):
        df, params = parse(fname)
        df['label'] = idx
        for k, v in params.items():
            df[k] = v
        yield df
        
df_all = pd.concat(_fn())

display(df_all.head())

In [ ]:
experiments = sorted(
    [
        x
        for x in df_all["experiment"].unique()
        if x.startswith("mk-rec") or x.startswith("mk-ran")
    ]
)
pprint(experiments)

In [ ]:
df_all.experiment.nunique()

In [ ]:
config_filenames = [c.split(".")[0] for c in config_filenames]
df_all = df_all[df_all.experiment.isin(config_filenames)]

In [ ]:
ax0.axhline?

In [ ]:
pd.Series(np.random.randn(5)).rolling(1).mean()

In [ ]:
import matplotlib.colors as mcolors

In [ ]:
pprint([x for x in dir(mcolors) if x.endswith("_COLORS")])

In [ ]:
mcolors.CSS4_COLORS

In [ ]:
pd.Series(np.random.randn(5)).rolling

In [ ]:
from typing import NamedTuple

In [ ]:
best_values

In [ ]:
metric = "HR@10"


def get_best_perfs(df_all: pd.DataFrame, exclude: Optional[List[str]]=None) -> pd.DataFrame:
    def _g():
        for experiment, df in df_all.groupby("experiment"):
            if exclude and any(s in experiment for s in exclude):
                continue
            best = df[metric].max()
            best_idx = df[df[metric] == best].index[0]
            best_values[experiment] = BestPerf(
                experiment=experiment, value=best, epoch=best_idx
            )
            yield {"experiment": experiment, metric: best, "epoch": best_idx}
    return pd.DataFrame(_g()).set_index('experiment')

df_best = get_best_perfs(df_all, exclude=['-sd-', '-mn-'])
df_best

In [ ]:
from matplotlib.markers import MarkerStyle

In [ ]:
MARKERS = list(MarkerStyle.markers.keys())

In [ ]:
import seaborn as sns

fig, ax = plt.subplots(figsize=(8, 8))
markers = dict(zip(df_best.index, MARKERS[: df_best.shape[0]]))
sns.scatterplot(
    y=metric,
    x="epoch",
    hue="experiment",
    data=df_best.reset_index(),
#     legend='brief',
    markers=markers,
    ax=ax,
    s=900,
    marker=".",
)
ax.legend(bbox_to_anchor=(1,1))
ax.grid(color="gainsboro", ls=":")
highest_epoch = df_best['epoch'].max()
lowest_epoch = df_best['epoch'].min()
ax.set_xlim(
    xmax=min(220, highest_epoch + 0.35 * highest_epoch),
    xmin=lowest_epoch - 0.35 * lowest_epoch,
)
highest_metric = df_best[metric].max()
lowest_metric = df_best[metric].min()
ax.set_ylim(
    ymax=highest_metric + 0.01 * highest_metric,
    ymin=lowest_metric - 0.01 * lowest_metric,
)


In [ ]:
list(list(np.random.randn(5)))

In [ ]:
fig, ax0 = plt.subplots(figsize=(10, 6), nrows=1, sharex=True)
criteria = "experiment in @experiments"
exclude = ["sd", "mk-ran", "se0100send1"]
for label, df in df_all.query(criteria).groupby("experiment"):
    if any(s in label for s in exclude):
        continue

    label_short = (
        label.replace("mk-", "")
        .replace("ca-", "")
        .replace("bs32ep200li005lu005nf10rg001", "")
        .replace("rvT", "")
    )
    df_ = df.set_index("epoch")
    _df_ran = df_all.query('experiment == "mk-ran-bs32ep200li005lu005nf10rg001"')[
        "HR@10"
    ]
    _df = 100 * ((df["HR@10"] - _df_ran) / _df_ran)
    first_max_loc = df_[df_["HR@10"] == df_["HR@10"].max()].index[0]
    #     _df.loc[:first_max_loc].rolling(10).max().plot(ax=ax0, label=label_short, alpha=0.9)
    _df.loc[:first_max_loc].plot(ax=ax0, label=label_short, alpha=0.9, lw=5)
    ax0.axhline(y=_df.max(), xmax=first_max_loc, lw=0.5, ls="dashed")
    #     df.set_index("epoch")["HR@10"].plot(ax=ax0, label=label_short, alpha=0.9)

    #     pd.DataFrame(summaries_dict["RandomSampler"])["HR@10"].plot(
    #         ax=ax0, label="From NB", color="red"
    #     )
    ax0.set_ylabel("HR@10")

#     ax1.set_ylim(0, 1)
#     df.set_index("epoch")["loss_avg"].plot(ax=ax1, label=label_short, alpha=0.9)
#     pd.DataFrame(summaries_dict["RandomSampler"])["loss_avg"].plot(
#         ax=ax1, label="From NB", color="red"
#     )
#     ax1.set_ylabel("Avg Loss")
plt.gca().legend()
fig.tight_layout()
ax0.set_xlim(0, 200)
ax0.grid(color="gainsboro", alpha=0.9, ls=":")
ax0.axhline(y=0, color="black")
# ax1.grid(color='gainsboro', alpha=0.7, ls=":")
plt.title("Percentage gain over RandomSampler's HR@10")
fig.tight_layout()

In [ ]:
Try a different model
Write out algorithm
SD and MN are the same
